In [1]:
""" 
Run nSimplices + MDS vs MDS on subset (STOOL and VAGINA only) of HMP dataset 
"""

' \nRun nSimplices + MDS vs MDS on subset (STOOL and VAGINA only) of HMP dataset \n'

In [1]:
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import os
import pandas as pd
import random as alea
from scipy.spatial.distance import pdist, squareform
from sklearn.manifold import MDS
import plotly.express as px


data_dir = "../data/"
output_dir = "../outputs"

target_sites = ['NOSE']
target_id = "N"
target_colors = ["blue"]

# target_sites = ['NOSE', 'VAGINA', 'THROAT']
# target_id = "NVT"
# target_colors = ["darkgreen", "orange", "deeppink"]

# target_sites = ['NOSE', 'THROAT']
# target_id = "NT"
# target_colors = ["blue", "orange"]

# if color == "deeppink":
#         return "THROAT"
#     if color == "black":
#         return "EARS"
#     if color == "cornflowerblue":
#         return "STOOL"
#     if color == "darkgreen":
#         return "NOSE"
#     if color == "red":
#         return "ELBOWS"
#     if color == "gray":
#         return "MOUTH"
#     if color == "orange":
#         return "VAGINA"

# target_sites = ['THROAT']
# target_id = "T"
# target_colors = ["deeppink"]


In [2]:
"""
Prepare dataset
"""

# iterate over QE and NB
files = ["hmp_v13lqphylotypeQuantNB_rs_c.csv", "hmp_v13lqphylotypeQuantE_rs_c.csv"]
color_df = pd.read_csv(os.path.join(data_dir, "hmp_v13lqphylotypePheno_rs_c.csv"), header=0)

for file in files:
    data_path = os.path.join(data_dir, file)
    df_hmp_ori = np.loadtxt(data_path, delimiter=",")
    df_hmp = []


    for index, row in color_df.iterrows():
        site_exist = False
        for site in target_sites:
            if row[site]:
                site_exist = True
        if site_exist:
            df_hmp.append(df_hmp_ori[index])
    df_hmp = np.array(df_hmp)
    np.savetxt(os.path.join(data_dir, file[:-8]+"target_"+target_id+"_"+file[-8:]), df_hmp, fmt='%1.17f', delimiter=',')


In [3]:
""" 
Prepare colors
"""

colors = []
new_color_df = color_df.copy(deep = True)
drop_indices = []

for index, row in color_df.iterrows():
    site_exist = False
    for i in range(len(target_sites)):
        site = target_sites[i]
        if row[site]:
            colors.append(target_colors[i])
            site_exist = True
    if not site_exist:
        drop_indices.append(index)

colors = np.array(colors)
print(colors.shape)

np.savetxt(os.path.join(data_dir, "hmp_target_"+target_id+"_"+"colors.txt"), colors, fmt="%s")

new_color_df = new_color_df.drop(drop_indices)
new_color_df.to_csv(os.path.join(data_dir, "hmp_v13lqphylotypePheno_target_"+target_id+"_"+"rs_c.csv"), header=True, index=False)

(136,)


In [4]:
""" 
Run nSimplices on HMP dataset
"""
colors = np.loadtxt(os.path.join(data_dir, "hmp_target_"+target_id+"_"+"colors.txt"), dtype="str")
exec(open("../nsimplices.py").read())
alea.seed(42)


In [5]:
""" 
Run
(1) NB normalization + nSimplices + cMDS 
(2) QE normalization + nSimplices + cMDS 

To derive the axes data
""" 

# output_files = ["hmp_target_"+target_id+"_"+"NB_nSimplices_cMDS_axes.txt", "hmp_target_"+target_id+"_"+"QE_nSimplices_cMDS_axes.txt"]
# data_files = ["hmp_v13lqphylotypeQuantNB_target_"+target_id+"_"+"rs_c.csv", "hmp_v13lqphylotypeQuantE_target_"+target_id+"_"+"rs_c.csv"]
output_files = ["hmp_target_"+target_id+"_"+"QE_nSimplices_cMDS_axes.txt"]
data_files = ["hmp_v13lqphylotypeQuantE_target_"+target_id+"_"+"rs_c.csv"]
subspace_dims = []
outlier_indices_list = []
for i in range(len(output_files)):
    output_file = output_files[i]
    data_file = data_files[i]
    axes_output_path = os.path.join(output_dir, output_file)
    print("======== NB/QE normalization + nSimplices + cMDS ========")
    data_path = os.path.join(data_dir, data_file)
    df_hmp = np.loadtxt(data_path, delimiter=",")
    hmp_dis_sq=squareform(pdist(df_hmp))

    feature_num = df_hmp.shape[1]
    dim_start = 1
    dim_end = 10
    # dim_start = 2
    # dim_end = 2

    print("hmp_dis_sq shape is:", hmp_dis_sq.shape)
    outlier_indices, subspace_dim , corr_pairwise_dis, corr_coord = nsimplices(hmp_dis_sq, feature_num, dim_start, dim_end, std_multi=3)
    print("subspace dimension is:", subspace_dim)
    subspace_dims.append(subspace_dim)
    outlier_indices_list.append(outlier_indices)

    # run cMDS to get the corrected coordinates in importance decreasing order
    _, _, Xe = cMDS(corr_pairwise_dis)
    np.savetxt(axes_output_path, Xe, fmt='%f')


======== NB/QE normalization + nSimplices + cMDS ========
hmp_dis_sq shape is: (136, 136)
dim in find_subspace_dim is: 1
dim in find_subspace_dim is: 2
dim in find_subspace_dim is: 3
dim in find_subspace_dim is: 4
dim in find_subspace_dim is: 5
dim in find_subspace_dim is: 6
dim in find_subspace_dim is: 7
dim in find_subspace_dim is: 8
dim in find_subspace_dim is: 9
dim in find_subspace_dim is: 10
med_height is: [1.49412392 1.3569765  1.22504754 1.19197936 1.11857634 1.06985826
 1.02103141 0.95719035 0.91758108 0.86791912]
subspace_dim one is: 3
thres is: 2.1864322409138968 mean is: 1.329537093949339 std is: 0.28563171565485257
outlier indices are: [ 23  40  42  62  89 108 112 122 135]
idx is: 23 height is: 2.4900245288498577 thres is: 2.1864322409138968
idx is: 40 height is: 2.534560936929293 thres is: 2.1864322409138968
idx is: 42 height is: 2.380984546339898 thres is: 2.1864322409138968
idx is: 62 height is: 2.372006087536816 thres is: 2.1864322409138968
idx is: 89 height is: 2.8109

/home/wanxinli/.local/lib/python3.9/site-packages/sklearn/manifold/_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


outlier_indices is: [ 23  40  42  62  89 108 112 122 135]
original coord is: [-0.12378256  0.10884893  0.10518448  0.00123769  0.15183973 -0.09071453
 -0.06620544  0.0191725  -0.1210875  -0.0687029  -0.1627812  -0.11371932
 -0.19458439  0.07080158  0.15158637  0.00771477 -0.01316919  0.04087977
  0.06740322  0.03681508  0.23004636  0.24763281 -0.01072502  0.06570144
 -0.15772351 -0.01487052  0.14127846  0.0460256   0.17251377 -0.06278738
  0.05120675  0.09841174  0.08555523  0.03257813  0.13576988 -0.07219419
 -0.19850861 -0.01226109 -0.18312864 -0.12612636 -0.17260817 -0.14457062
 -0.18308108 -0.03013555  0.16661697  0.02447087  0.13556928 -0.09830715
  0.11631766 -0.0873592  -0.12969755 -0.06972097  0.15270875  0.02349758
  0.02919644  0.04599559  0.14317627 -0.07465266 -0.20430556 -0.00623789
 -0.02377179  0.08891246  0.14385268 -0.08021412  0.15110597 -0.07601674
  0.05584559  0.22655085 -0.12374931  0.09902986  0.18811352  0.13376126
  0.09042434 -0.02749888 -0.08681658 -0.2407774

In [6]:
""" 
Analyze the number of outliers
"""
print("subspace_dim is:", subspace_dim)
print("outlier number is:", len(outlier_indices_list[0]))
print("outlier percent is:", len(outlier_indices_list[0])/len(colors))

subspace_dim is: 3
outlier number is: 9
outlier percent is: 0.0661764705882353


In [7]:
# """ 
# Plot distribution of dimensions over the range of candidate dimensions 
# only for NB normalization
# """
# data_file  = "hmp_v13lqphylotypeQuantNB_target_"+target_id+"_"+"rs_c.csv"
# data_path = os.path.join(data_dir, data_file)
# df_hmp = np.loadtxt(data_path, delimiter=",")
# hmp_dis_sq=squareform(pdist(df_hmp))

# num_point = hmp_dis_sq.shape[0]
# hcolls = []
# start_dim = 1
# end_dim = 50
# for dim in range(start_dim, end_dim+1):
#     print("current dimension is:", dim)
#     heights = nsimplices_all_heights(num_point, hmp_dis_sq, dim, seed=dim+1)
#     hcolls.append(heights)

# # calculate median heights for tested dimension from start_dim to end_dim
# h_meds = []
# for hcoll in hcolls:
#     h_meds.append(np.median(hcoll))

# # calculate the ratio, where h_med_ratios[i] corresponds to h_meds[i-1]/h_meds[i]
# # which is the (median height of dim (i-1+start_dim))/(median height of dim (i+start_dim))
# h_med_ratios = []
# for i in range(1, len(hcolls)):
#     h_med_ratios.append(h_meds[i-1]/h_meds[i])

# # plot the height scatterplot and the ratios
# plt.figure(0)
# fig, ax1 = plt.subplots()
# color = 'red'
# ax1.set_xlabel(r'dimension tested $n$', fontsize=15)
# ax1.set_ylabel(r'median of heights', color = color, fontsize=15)
# ax1.scatter(list(range(start_dim, end_dim+1)), h_meds, color = color, s=10)
# ax1.plot(list(range(start_dim, end_dim+1)), h_meds, color=color)
# ax1.tick_params(axis ='y', labelcolor = color)
 
# # Adding Twin Axes to plot using dataset_2
# ax2 = ax1.twinx()
 
# color = 'black'
# ax2.set_ylabel(r'heights median ratio: $h_{n-1}/h_n$', color = color, fontsize=15)
# ax2.plot(list(range(start_dim+1, end_dim+1)), h_med_ratios, color = color)
# ax2.tick_params(axis ='y', labelcolor = color)
# plt.tight_layout(pad=2)
 
# # Show plot
# plt.savefig(os.path.join(output_dir, "hmp_"+target_id+"_dim.png"))

In [8]:
# print(h_med_ratios)

In [9]:
""" 
Inferred dimension
QE: STOOL - 3, VAGINA - 3, THROAT - 3, EARS - 3, NOSE - 3, ELBOWS - 3, MOUTH - 3
NB: STOOL - 41, VAGINA - 5, THROAT - , EARS - 3, NOSE - 3, ELBOWS - 3, MOUTH - 3

"""

' \nInferred dimension\nQE: STOOL - 3, VAGINA - 3, THROAT - 3, EARS - 3, NOSE - 3, ELBOWS - 3, MOUTH - 3\nNB: STOOL - 41, VAGINA - 5, THROAT - , EARS - 3, NOSE - 3, ELBOWS - 3, MOUTH - 3\n\n'

In [10]:
print("number of samples is:", len(colors))
print("number of outliers is:", len(outlier_indices_list[0]))

number of samples is: 136
number of outliers is: 9


site - number of samples - number of outliers - total distance (nSimplices) - total distance (MDS)

NOSE: 136 - 18 - 0.9809061157393936 - 1.2269812549552055

NOSE, THROAT: 270 - 30 - 1.362760825075196 - 1.3811090861305209

THROAT: 134 - 14

VAGINA: 186 - 26

VAGINA, STOOL, THROAT: 454 - 25


In [11]:
""" 
Run
(1) NB normalization + cMDS 
(2) QE normalization + cMDS 

To derive the axes data
"""  

axes_files = ["hmp_target_"+target_id+"_"+"NB_MDS_cMDS_axes.txt", "hmp_target_"+target_id+"_"+"QE_MDS_cMDS_axes.txt"] # put NB before QE
data_files = ["hmp_v13lqphylotypeQuantNB_target_"+target_id+"_"+"rs_c.csv", "hmp_v13lqphylotypeQuantE_target_"+target_id+"_"+"rs_c.csv"]

for i in range(len(axes_files)):
    axes_file = axes_files[i]
    data_file = data_files[i]

    print("======== QE/NB normalization + MDS + cMDS ========")
    data_path = os.path.join(data_dir, data_file)
    axes_output_path = os.path.join(output_dir, axes_file)

    df_hmp = np.loadtxt(data_path, delimiter=",")
    hmp_dis_sq=squareform(pdist(df_hmp))

    # Plot cMDS embedding using the pairs of axis from the four most significant axes 
    # enforce_dim = subspace_dims[i] # enforcing the dimension to be consistent with nSimplices QE+nsimplices+cMDS
    feature_num = df_hmp.shape[1]
    print("feature_num is:", feature_num)
    embedding = MDS(n_components=feature_num, max_iter=100000000000, dissimilarity='precomputed') 
    corr_coord = embedding.fit_transform(hmp_dis_sq)
    corr_dis_sq=squareform(pdist(corr_coord))
    _, _, Xe = cMDS(corr_dis_sq)

    np.savetxt(axes_output_path, Xe, fmt='%f')

======== QE/NB normalization + MDS + cMDS ========
feature_num is: 425


/home/wanxinli/.local/lib/python3.9/site-packages/sklearn/manifold/_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


======== QE/NB normalization + MDS + cMDS ========
feature_num is: 425


/home/wanxinli/.local/lib/python3.9/site-packages/sklearn/manifold/_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


In [12]:

""" 
Compute the average distance between the outlieres and the barycenter of regular points
"""

def centeroid(coords_list):
    """ 
    Computes the barycenter of a list of coordinates in coords_lst
    """

    length = coords_list.shape[0]
    centroid = []
    for i in range(coords_list.shape[1]):
        centroid.append(np.sum(coords_list[:, i])/length)
    return np.array(centroid)
    
axes_files = ["hmp_target_"+target_id+"_"+"QE_nSimplices_cMDS_axes.txt", \
    "hmp_target_"+target_id+"_"+"QE_MDS_cMDS_axes.txt"]

for i in range(len(axes_files)):
    axes_file = axes_files[i]
    subspace_dim = subspace_dims[0] # TODO: this should be change to i%2
    outlier_indices =  outlier_indices_list[0] # TODO: this should be change to i%2
    Xe = np.loadtxt(os.path.join(output_dir, axes_file))
    normal_indices = list(set(range(Xe.shape[0])) - set(outlier_indices))
    normal_centroid = centeroid(Xe[normal_indices, :]) # should only focus on the first inferred dimension coordinates
    normal_centroid = normal_centroid[:subspace_dim]
    outlier_Xe = Xe[outlier_indices, :subspace_dim]
    
    # compute distance
    total_distance = 0
    for outlier_coord in outlier_Xe:
        total_distance += np.linalg.norm(outlier_coord-normal_centroid)
    print("average distance is:", total_distance/len(outlier_indices))

    



average distance is: 0.9130198609268475
average distance is: 1.208428007655835


In [14]:
""" 
Plot pairwise result
"""

# figure_files = ["hmp_target_"+target_id+"_"+"NB_nSimplices_cMDS.png", \
#     "hmp_target_"+target_id+"_"+"QE_nSimplices_cMDS.png", \
#     "hmp_target_"+target_id+"_"+"NB_MDS_cMDS.png", \
#     "hmp_target_"+target_id+"_"+"QE_MDS_cMDS.png"]
# axes_files = ["hmp_target_"+target_id+"_"+"NB_nSimplices_cMDS_axes.txt", \
#     "hmp_target_"+target_id+"_"+"QE_nSimplices_cMDS_axes.txt", \
#     "hmp_target_"+target_id+"_"+"NB_MDS_cMDS_axes.txt", \
#     "hmp_target_"+target_id+"_"+"QE_MDS_cMDS_axes.txt"]
titles = ["QuantE+nSimplices", "QuantE+MDS"]
figure_files = [
    "hmp_target_"+target_id+"_"+"QE_nSimplices_cMDS.png", \
    "hmp_target_"+target_id+"_"+"QE_MDS_cMDS.png"]
axes_files = [
    "hmp_target_"+target_id+"_"+"QE_nSimplices_cMDS_axes.txt", \
    "hmp_target_"+target_id+"_"+"QE_MDS_cMDS_axes.txt"]
titles = ["QuantE+nSimplices",  "QuantE+MDS"]
num_axes = 3 # show pairwise 2D plot to decompose the 3D plot

for i in range(len(figure_files)):
    # if i == 0 or i == 2:
    #     continue
    figure_file = figure_files[i]
    axes_file = axes_files[i]
    title = titles[i]
    outlier_indices =  outlier_indices_list[0] # TODO: change to i%2

    print("======== plot pairwise 2D plot (subset) ========")
    Xe = np.loadtxt(os.path.join(output_dir, axes_file))
    print(Xe.shape)
    for first_dim in range(num_axes):
        for second_dim in range(first_dim+1, num_axes):
            plt.figure()
            
            target_indices = []
            for color in target_colors:
                cur_indices = [i for i, e in enumerate(colors) if e == color]
                target_indices.append(cur_indices)
            # stool_indices = [i for i, e in enumerate(colors) if e == 'cornflowerblue']
            # ears_indices = [i for i, e in enumerate(colors) if e == 'orange']

            for i in range(len(target_colors)):
                color = target_colors[i]
                site = target_sites[i]
                site_indices = target_indices[i]
                
                # if 'nSimplices' not in figure_file:
                #     plt.scatter(Xe[site_indices, second_dim], \
                #     Xe[site_indices, first_dim], s=5, c=color, label = site)
                # else:
                site_outlier_indices = list(set(site_indices) & set(outlier_indices))
                site_normal_indices = list(set(site_indices) - set(outlier_indices))
            
                plt.scatter(Xe[site_normal_indices, second_dim], \
                    Xe[site_normal_indices, first_dim], s=5, c=color, label = site)
                plt.scatter(Xe[site_outlier_indices, second_dim], \
                    Xe[site_outlier_indices, first_dim], s=15, c=color, label = site+" outlier", marker="x")
                    
            plt.legend()
            plt.title(title, size=10)   
            plt.savefig(os.path.join(output_dir, figure_file[:-4]+"_"+str(first_dim)+"_"+str(second_dim)+".png"))
            

======== plot pairwise 2D plot (subset) ========
(136, 131)
======== plot pairwise 2D plot (subset) ========
(136, 135)


In [15]:
def color_to_site(color):
    """
    Returns the site for the color
    """
    if color == "orange":
        return "THROAT"
    if color == "black":
        return "EARS"
    if color == "cornflowerblue":
        return "STOOL"
    if color == "blue":
        return "NOSE"
    if color == "red":
        return "ELBOWS"
    if color == "gray":
        return "MOUTH"
    if color == "yellow":
        return "VAGINA"

def site_to_color(site):
    """
    Returns the color for the site
    """
    color_discrete_map = {'STOOL': 'cornflowerblue', 'VAGINA': 'yellow', 'THROAT': 'orange',\
        "EARS": 'black', "NOSE": "blue", "ELBOWS": 'red', "MOUTH": 'grey' }

    return color_discrete_map[site]

In [17]:
""" 
generate 3D plot of the first three axes 
"""

# axes_files = ["hmp_target_"+target_id+"_"+"NB_nSimplices_cMDS_axes.txt", \
#     "hmp_target_"+target_id+"_"+"QE_nSimplices_cMDS_axes.txt", \
#     "hmp_target_"+target_id+"_"+"NB_MDS_cMDS_axes.txt", \
#     "hmp_target_"+target_id+"_"+"QE_MDS_cMDS_axes.txt"]
    

# figure_files = ["hmp_target_"+target_id+"_"+"NB_nSimplices_cMDS.html", \
#     "hmp_target_"+target_id+"_"+"QE_nSimplices_cMDS.html",\
#     "hmp_target_"+target_id+"_"+"NB_MDS_cMDS.html", \
#     "hmp_target_"+target_id+"_"+"QE_MDS_cMDS.html"]


axes_files = [
    "hmp_target_"+target_id+"_"+"QE_nSimplices_cMDS_axes.txt", \
    "hmp_target_"+target_id+"_"+"QE_MDS_cMDS_axes.txt"]
    

figure_files = [
    "hmp_target_"+target_id+"_"+"QE_nSimplices_cMDS.html",\
    "hmp_target_"+target_id+"_"+"QE_MDS_cMDS.html"]


for i in range(len(axes_files)):
    axes_file = axes_files[i]
    figure_file = figure_files[i]
    outlier_indices = outlier_indices_list[0] #TODO: change this to %2
    normal_indices = list(set(list(range(Xe.shape[0]))) - set(outlier_indices))
    Xe = np.loadtxt(os.path.join(output_dir, axes_file))
    color_discrete_map = {'STOOL': 'cornflowerblue', 'VAGINA': 'yellow', 'THROAT': 'orange',\
        "EARS": 'black', "NOSE": "blue", "ELBOWS": 'red', "MOUTH": 'grey' }


    Xe_df = pd.DataFrame(Xe[:,:3], columns = ["axis_0", "axis_1", "axis_2"])
    Xe_df['color'] = colors
    Xe_df['label'] = \
        Xe_df.apply(lambda row: color_to_site(row['color']), axis=1)
    outlier_indicator = np.array([0]*len(colors))
    outlier_indicator[outlier_indices] = 1
    Xe_df['outlier'] = outlier_indicator.tolist()

        # specify trace names and symbols in a dict
    symbols = {'1': 'cross',
            '0':'circle'}

    

    fig = px.scatter_3d(Xe_df, x='axis_0', y='axis_1', z='axis_2',
            color='label', color_discrete_map=color_discrete_map, symbol='outlier')

    for i, d in enumerate(fig.data):
        # fig.data[i].marker.symbol = symbols[fig.data[i].name] 
        print(symbols)
        fig.data[i].marker.symbol = symbols[fig.data[i].name.split(',')[1].strip()] 
        
    fig.for_each_xaxis(lambda axis: axis.title.update(font=dict(color = 'blue', size=30)))
    fig.update_layout(scene = dict(
                    xaxis_title='axis 0',
                    yaxis_title='axis 1',
                    zaxis_title='axis 2'))
    fig.update_scenes(xaxis_title_font=dict(size=50))
    fig.update_scenes(yaxis_title_font=dict(size=50))
    fig.update_scenes(zaxis_title_font=dict(size=50))
    fig.update_scenes(xaxis = dict(tickfont=dict(size=15)))
    fig.update_scenes(yaxis = dict(tickfont=dict(size=15)))
    fig.update_scenes(zaxis = dict(tickfont=dict(size=15)))
    fig.write_html(os.path.join(output_dir, figure_file))




{'1': 'cross', '0': 'circle'}
{'1': 'cross', '0': 'circle'}
{'1': 'cross', '0': 'circle'}
{'1': 'cross', '0': 'circle'}


In [16]:
# """
# Combine three 3D dynamic plots together
# https://stackoverflow.com/questions/59868987/plotly-saving-multiple-plots-into-a-single-html/59869358#59869358
# """
# import plotly.graph_objects as go
# import plotly.express as px
# import plotly.offline as offline
# import pandas as pd

# from plotly.subplots import make_subplots


# """ 
# generate 3D plot of the first three axes 
# """

# target_id = 'M'

# axes_files = ["hmp_target_"+target_id+"_"+"NB_nSimplices_cMDS_axes.txt", \
#     "hmp_target_"+target_id+"_"+"NB_MDS_cMDS_axes.txt"]
    

# figure_files = ["hmp_target_"+target_id+"_"+"NB_nSimplices_cMDS.html", \
#     "hmp_target_"+target_id+"_"+"NB_MDS_cMDS.html"]

# fig = make_subplots(\
#     rows=2, cols=1, shared_xaxes=True, \
#     vertical_spacing=0.02)



# for i in range(len(axes_files)):
#     axes_file = axes_files[i]
#     figure_file = figure_files[i]
#     outlier_indices = outlier_indices_list[i%2]
#     normal_indices = list(set(list(range(Xe.shape[0]))) - set(outlier_indices))
#     Xe = np.loadtxt(os.path.join(output_dir, axes_file))
#     color_discrete_map = {'STOOL': 'cornflowerblue', 'VAGINA': 'orange', 'THROAT': 'deeppink',\
#         "EARS": 'black', "NOSE": "darkgreen", "ELBOWS": 'red', "MOUTH": 'grey' }


#     Xe_df = pd.DataFrame(Xe[:,:3], columns = ["axis_0", "axis_1", "axis_2"])
#     Xe_df['color'] = colors
#     Xe_df['label'] = \
#         Xe_df.apply(lambda row: color_to_site(row['color']), axis=1)
#     outlier_indicator = np.array(['nor']*len(colors))
#     outlier_indicator[outlier_indices] = "out"
#     Xe_df['outlier'] = outlier_indicator.tolist()

#         # specify trace names and symbols in a dict
#     symbols = {'out': 'cross',
#             'nor':'circle'}

    

#     # fig = px.scatter_3d(Xe_df, x='axis_0', y='axis_1', z='axis_2',
#     #         color='label', color_discrete_map=color_discrete_map, symbol='outlier')


#     fig.add_trace(go.Scatter3d(x = Xe_df["axis_0"], y = Xe_df["axis_1"], z = Xe_df["axis_2"], \
#         marker=dict(
#             color = colors
#         )), \
#         row=1, col=1)

#     # for i, d in enumerate(fig.data):
#     #     # fig.data[i].marker.symbol = symbols[fig.data[i].name] 
#     #     print(symbols)
#     #     fig.data[i].marker.symbol = symbols[fig.data[i].name.split(',')[1].strip()] 

#     # fig.update_layout(scene = dict(
#     #                 xaxis_title='axis 0',
#     #                 yaxis_title='axis 1',
#     #                 zaxis_title='axis 2'))
#     fig.write_html(os.path.join(output_dir, figure_file))

# offline.plot(fig, filename='name.html')



